# Experimentation with fuzzy text matching
Many sessions with Claude have convinced me that the only way to match companies between the AFSC (or other) lists and the SEC data is through *text matching* of company names. There are just too many numbers and exchanges and share classes for stock.

## author:
- **David W. Hogg** (NYU)

## notes:
- Here's one relevant conversation with Claude: https://claude.ai/share/bde7bc8b-f8d5-40b6-aa0c-e5187dc314a5

## bugs:
- I just set the matching method and score threshold by hand?

In [ ]:
# !pip install rapidfuzz

In [ ]:
import os
import pandas as pd
from rapidfuzz import process, fuzz

In [ ]:
DATA_DIR = os.path.expanduser("../data")
scorer = fuzz.ratio # this is magic

In [ ]:
# idiotic testing written by Claude
long_list = ["Apple Inc.", "Microsoft Corporation", "Alphabet Inc.", ]
short_list = ["Microsoft Corp", "apple incorporated", ]
for query in short_list:
    match, score, index = process.extractOne(query, long_list, scorer=scorer)
    if score >= 50:  # tune this threshold
        print(f"{query} → {match} (score: {score})")

In [ ]:
bad_list = pd.read_csv(os.path.join(DATA_DIR, "investigate-dataset-july-2025.csv"))
print(bad_list.columns)

In [ ]:
fund_list = pd.read_csv(os.path.join(DATA_DIR, "Nuveen_Large_Cap_Value_Fund.csv"))
print(fund_list.columns)

In [ ]:
for query in bad_list["Company Standard Name"]:
    match, score, index = process.extractOne(query, fund_list["ISSUER_NAME"], scorer=scorer)
    if score >= 70:  # tune this threshold
        print(f"{query} → {match} (score: {score})")

In [ ]:
# okay, it looks like the only issues are instances of "The".
# let's look for those
column, substring = "ISSUER_NAME", "The"
foo = fund_list[fund_list[column].str.contains(substring, na=False)][column]
print(substring, len(foo), "\n", foo)
column = "Company Standard Name"
bar = bad_list[bad_list[column].str.contains(substring, na=False)][column]
print(substring, len(bar), "\n", bar)

In [ ]:
# look for other possible issues -- Corporation, Company, International, etc.
substring = "Incorporated"
column = "ISSUER_NAME"
foo = fund_list[fund_list[column].str.contains(substring, na=False)][column]
print(substring, len(foo), "\n", foo)
column = "Company Standard Name"
bar = bad_list[bad_list[column].str.contains(substring, na=False)][column]
print(substring, len(bar), "\n", bar)